In [ ]:
import json,csv
from kafka import KafkaProducer
import time

In [ ]:
def sender(file,producer,sleep_time=0.00015):
    path ='./Data/'
    with open(path+file+'.csv') as file:
        reader = csv.DictReader(file, delimiter=",")
        i = 0
        start = time.time()
        for row in reader:
            
            row["HEAD"] = int(row["HEAD"])
            row["FPGA"] = int(row["FPGA"])
            row["TDC_CHANNEL"] = int(row["TDC_CHANNEL"])
            row["ORBIT_CNT"] = float(row["ORBIT_CNT"])
            row["BX_COUNTER"] = int(row["BX_COUNTER"])
            row["TDC_MEAS"] = float(row["TDC_MEAS"])
            producer.send(topic='Experiment_measurements', value=row, )
            #producer.send(topic='Experiment_measurements', value=json.dumps(row))
            #producer.flush()
            time.sleep(sleep_time) #This was adjusted "by eye" to send 1000 elements/second
            i += 1
            if (i==1000):
                i = 0
                end = time.time()
                print(f"The time to print elements is:",end-start)
                start = end
                

In [ ]:
KAFKA_BOOTSTRAP_SERVERS = 'slave01:9092'
#KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
producer = KafkaProducer( bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                               value_serializer=lambda v: json.dumps(v).encode('utf-8'))

for file in ['data_000000']:
    print(file)
    sender(file, producer,0.0007)

In [ ]:
producer.close()